<a href="https://colab.research.google.com/github/woodwj/TD_mlproject/blob/Model_1/TDML_model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 1 - Investigations

In [ ]:
!pip install yfinance # collect packages

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


**Model 1 - Normal Equations; predicting stock closing prices from opening prices**

In [ ]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.metrics import mean_absolute_percentage_error

class dataset:
  
    def fetch(self, stock, period):
        d = yf.download(stock, period=period)
        return d["Open"].to_numpy().reshape(d.shape[0],1), d["Close"].to_numpy().reshape(d.shape[0],1)

    def tilda(self,data):
        return np.concatenate([np.ones((data.shape[0],1)), data],axis=1)

    # use "period" instead of start/end
    # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
    # (optional, default is '1mo')
    def __init__(self,stocks,period):
        X,Y = self.fetch(stocks, period)
        split = int(len(X)*0.8)
        X_tr = X[:split]
        X_te = X[split:]
        self.Y_tr = Y[:split]
        self.Y_te = Y[split:]
        self.X_te = self.tilda(X_te)
        self.X_tr = self.tilda(X_tr)
        self.X_val = self.tilda(X)
        self.Y_val = Y
        
class pen_inv_model:

    def __init__(self, dataset):
        self.data = dataset

    def train(self):
        self.W = np.linalg.pinv(self.data.X_tr) @ self.data.Y_tr

    def f(self,X_t):
        return  X_t @ self.W

    def error(self):
        return self.data.Y_te - self.f(self.data.X_te)

    def RMSE(self):
        return np.sqrt(np.mean((self.error())**2))

    def MAPE(self):
      return mean_absolute_percentage_error(self.data.Y_te,self.f(self.data.X_te))


In [ ]:
def experiment(stocks, periods):

  for stock in stocks.split():
    print(f"\nRESULTS FOR {stock}\n")
    RMSE_scores = {}
    MAPE_scores = {}

    for period in periods:
      print("PERIOD:"+period)
      d = dataset(stock,period)
      model = pen_inv_model(d)
      model.train()
      y_prediction = model.f(d.X_te)

      RMSE_scores[period] = model.RMSE()
      MAPE_scores[period] = model.MAPE()

      # plot predicted against input with actual
      pred, = plt.plot(d.X_te[:,1:], y_prediction, c='crimson')
      pred.set_label('Line of regression')
      actu= plt.scatter(d.X_te[:,1:], d.Y_te, c ='blue')
      actu.set_label('Actual Closing Prices')
      plt.legend()
      plt.xlabel('Opening Price')
      plt.ylabel('Closing Price')
      plt.tight_layout()
      plt.show()

    # d is max period dataset
    plot_acf(d.X_te[:,1:], lags=40)
    plt.show()

    plt.plot(list(RMSE_scores.keys()),list(RMSE_scores.values()))
    plt.grid()
    plt.title(f"RMSE error for {stock}")
    plt.show()

    plt.plot(list(MAPE_scores.keys()),list(MAPE_scores.values()))
    plt.grid()
    plt.title(f"MAPE error for {stock}")
    plt.show()

# valid periods: 5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
periods = ("5d","1mo","3mo","6mo","1y","2y","5y","10y","max")
experiment("BP AAPL MSFT",periods)

Error is strongly positvely corelated in timescale and error metrics with max data shows the effect of this corelation. Scatter graphs thus misleading with scale.

Scatter graphs show our line of regression is too linear.  Next will add l2 regularization to stop this underfitting. Evaluating λ hyperameter and comparing our new model to the previous.